In [2]:
import pandas as pd
import psycopg2
import numpy as np
import sklearn
import pyodbc

In [54]:
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")

# usage: specify file location, sample size and seed(used by random)
filepath = 'train.tsv'
filepathTest = 'test.tsv'

#filepath = 'news_sample.csv' # <- overwrite for setup
s = 200                    # desired sample size(seems to have slack ie. not exact)
stest = 200                    # desired sample size(seems to have slack ie. not exact)

seed = 1                     # seed used by Pseudorandom number generator

df_train = pd.read_csv(filepath, header = 0, sep='\t').sample(n=s, random_state=seed)
df_train["content"] = df_train["content"].astype(str)

df_test = pd.read_csv(filepathTest, header=0, sep='\t').sample(n=stest, random_state=seed)
df_train["content"] = df_train["content"].astype(str)

df_test = df_test.reset_index(drop=True)
df_test = df_train.reset_index(drop=True)

# df = pd.read_sql_query("""Select content, type_id
# from article""", connection)

In [53]:
df_test.head()

,idjson,rating,content,tags,author,position,state,party,number1,number2,number3,number4,number5,media
0,5749.json,half-true,Says his Texas election fraud investigations h...,"crime,elections,legal-issues",greg-abbott,governor,Texas,republican,11,3,8,6,5,a press release
1,12858.json,half-true,"Thanks to the Trans-Pacific Partnership, Vietn...","labor,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference at the White House
2,7448.json,true,Felony crimes in the city of Atlanta are the l...,crime,kasim-reed,NaN,NaN,democrat,1,0,5,7,1,a speech
3,7886.json,mostly-true,We havent had a Republican senator in Washingt...,"congress,history",christine-whitman,"President, The Whitman Strategy Group",New Jersey,republican,0,0,0,2,0,an interview on The John Gambling radio show
4,10175.json,mostly-true,The gender gap in his support is not unique to...,"elections,women",scott-walker,Milwaukee County Executive,Wisconsin,republican,26,41,32,40,11,an interview


In [45]:
# VIGTIG NOTE: måden jeg opdeler sættet på er MEGET dårlig, på det endelige skal vi shuffle og derefter opdele! 

In [ ]:
### Nearest hood (neighbors)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df["content"])

NameError: name 'df' is not defined

In [47]:
from sklearn.neighbors import NearestCentroid

X = np.array(x.toarray()[int(len(df["type_id"])/2):])
y = np.array(df["type_id"][int(len(df["type_id"])/2):])
clf = NearestCentroid()
clf.fit(X, y)

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [48]:
test_X = x.toarray()[:int(len(df["type_id"])/2)]
test_y = df["type_id"][:int(len(df["type_id"])/2)]

predictions = clf.predict(np.array(test_X))

failCounter = 0
for i in range(int(len(df["type_id"])/2)):
    if (predictions[i] != test_y[i]):
        failCounter += 1
        
print("Number of mislabeled points out of a total %d points : %d" % (len(test_X), failCounter))

Number of mislabeled points out of a total 125 points : 63


In [29]:
### Naive bayes

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(x.toarray(), df["type_id"], test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 125 points : 57


In [42]:
### Decision trees

In [61]:
from sklearn import tree

X = x.toarray()[int(len(df["type_id"])/2):]
y = df["type_id"][int(len(df["type_id"])/2):]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_X, train_y)

In [62]:
test_X = x.toarray()[:int(len(df["type_id"])/2)]
test_y = df["type_id"][:int(len(df["type_id"])/2)]

predictions = clf.predict(np.array(train_X))

failCounter = 0
for i in range(int(len(df["type_id"])/2)):
    if (predictions[i] != test_y[i]):
        failCounter += 1
        
print("Number of mislabeled points out of a total %d points : %d" % (len(test_X), failCounter))

Number of mislabeled points out of a total 125 points : 0


In [63]:
### Support Vector Machines

In [67]:
from sklearn import svm
X = x.toarray()[int(len(df["type_id"])/2):]
y = df["type_id"][int(len(df["type_id"])/2):]
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [66]:
### note testen gik meget godt, men det betyder ikke man har en god model ;)
clf.predict(np.array(train_X))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [68]:
test_X = x.toarray()[:int(len(df["type_id"])/2)]
test_y = df["type_id"][:int(len(df["type_id"])/2)]

predictions = clf.predict(np.array(train_X))

failCounter = 0
for i in range(int(len(df["type_id"])/2)):
    if (predictions[i] != test_y[i]):
        failCounter += 1
        
print("Number of mislabeled points out of a total %d points : %d" % (len(test_X), failCounter))

Number of mislabeled points out of a total 125 points : 61


In [8]:
# Med Lair datasæt

In [ ]:
### nearest hood

In [70]:
X_train, X_test, y_train, y_test = train_test_split(x.toarray(), df_train["rating"], test_size=0.5, random_state=0)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df_train["content"])

In [77]:
from sklearn.neighbors import NearestCentroid

clf = NearestCentroid()
clf.fit(X_train, y_train)

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [ ]:
### Naive bias

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 100 points : 78


In [68]:
### Decision trees

In [90]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [ ]:
### Support Vector Machine

In [93]:
from sklearn import svm

clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
### test model

In [94]:
predictions = clf.predict(np.array(test_X))

failCounter = 0
for i in range(int(len(df_test["rating"])/2)):
    if (predictions[i] != test_y[i]):
        failCounter += 1
        
print("Number of mislabeled points out of a total %d points : %d" % (len(test_X), failCounter))

Number of mislabeled points out of a total 100 points : 34


In [96]:
from sklearn.metrics import f1_score

f1_score(y_test, predictions, average='macro')

0.15919645370109767